# Home2
Your home away from home <br>
The best location for your needs, anywhere in the world <br>
### Inputs: 
    places (eg. 'Pune, Maharashtra')
    Category List (eg. 'Food', 'Restaurant', 'Gym', 'Trails', 'School', 'Train Station')
    Limit of Results to return (eg. 75)
    Radius of search in metres (eg. 10,000)
    Radiums of hotels to search
### Outputs:
    Cluster of venues coded by criteria
    map of cluster
    Centroid latitude and longitude for these venues
    place near centroid
    Hotels near centroid

## User Input

In [6]:
# places to analyze venues around and obtain best location
places=['Bend, Oregon'] 
# 4square Venue categories of interest (https://developer.foursquare.com/docs/build-with-foursquare/categories)
categories=['museum','thai restaurant','gym','night market','national park','state/provincial park','trail']  

## Import Libraries

In [7]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import math

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an place into latitude and longitude values

import requests # library to handle requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
 

print('Libraries imported.')

Libraries imported.


In [8]:
# Limit of search results  
LIMIT=50
radius,hotel_radius=20000,20000
centroid_mean_median='median'
# Radius of search in km (maximum 100)
radius=20
# Radius in km to search hotels around from final optimum location (centroid). This list is also sorted by likes    
hotel_radius=20 

## Functions and Classes

In [9]:
def build_tree(data, parent=None):
    from anytree import Node, RenderTree
    # Define the root node inside the function
    root = Node("root")
    def _build_tree(data, parent):
        if isinstance(data, dict):
            for key, value in data.items():
                node = Node(key, parent=parent)
                _build_tree(value, node)
        elif isinstance(data, list):
            for index, item in enumerate(data):
                node = Node(f'Item {index}', parent=parent)
                _build_tree(item, node)
        else:
            Node(data, parent=parent)
    # Build the tree from the JSON data
    _build_tree(data, root)
    # Print the tree
    for pre, fill, node in RenderTree(root):
        print(f"{pre}{node.name}")
        
def extract_name_id_pairs(dictionary):
    var = {}
    def explore_dict(d):
        if isinstance(d, dict):
            if 'name' in d and 'id' in d:
                var[d['name'].lower()] = d['id']
            for key, value in d.items():
                explore_dict(value)
        elif isinstance(d, list):
            for item in d:
                explore_dict(item)
    explore_dict(dictionary)
    return var


<b> Function to get centroids from a df of places </b>

In [10]:
def centroids(place_df):
    ''' Returns the centroid of a df of places as a dictionary of place:(lat,long)'''
    if centroid_mean_median=='mean':
        centroid_dict={place:(place_df[place_df.Address==place]['Venue Latitude'].mean(),place_df[
            place_df.Address==place]['Venue Longitude'].mean()) for place in places}
    else:
        centroid_dict={place:(place_df[place_df.Address==place]['Venue Latitude'].median(),place_df[
            place_df.Address==place]['Venue Longitude'].medkan()) for place in places}
    return centroid_dict

## Obtain Location and Venue information in a Dataframe

### Create Geolocator using Nominatim and Obtain Location info. for the places

In [11]:
geoagent="explorer"

lat=[]
long=[]
for place in places:
    geolocator = Nominatim(user_agent=geoagent)
    loc = geolocator.geocode(place)
    lat.append(loc.latitude)
    long.append(loc.longitude)
    print('The geograpical coordinates of '+place +' are {}, {}.'.format(lat[-1], long[-1]))



The geograpical coordinates of Bend, Oregon are 44.0581728, -121.3153096.


In [12]:
df_loc=pd.DataFrame({'Name': places,'Latitude': lat, 'Longitude':long})
df_loc

,Name,Latitude,Longitude
0,"Bend, Oregon",44.058173,-121.31531


### Foursquare Credentials

In [13]:
CLIENT_ID = 'KB1MA3E55GWTISTXW0GT5LZU1IXBUUDU4VW4QXSKHOQDEVMP' # your Foursquare ID
CLIENT_SECRET = '33L0T0LN54ETAGYXW0W41JCF5CLWHNOUCROOE4NMCUWHTZQA' # your Foursquare Secret
VERSION = '20240725' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KB1MA3E55GWTISTXW0GT5LZU1IXBUUDU4VW4QXSKHOQDEVMP
CLIENT_SECRET:33L0T0LN54ETAGYXW0W41JCF5CLWHNOUCROOE4NMCUWHTZQA


### Make list of category names by ID

In [14]:
categories_url='https://api.foursquare.com/v2/venues/categories?&client_id={}&client_secret={}&v={}'.format(CLIENT_ID,
                                                                                                            CLIENT_SECRET,
                                                                                                            VERSION)
result=requests.get(categories_url).json()
# build_tree(resut)
# Build the tree from the JSON data
cat_info=result['response']['categories']

category_dict=extract_name_id_pairs(cat_info)
categories=[category_dict[cat.lower()] for cat in categories if cat in list(category_dict.keys()) ]
cat_dict = {value: key for key, value in category_dict.items()}

In [15]:
# General Search URL string
url_str='https://api.foursquare.com/v2/venues/search?categoryId={}&client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'
# Zipcode Search URL string
url_str_zip='https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'

### Explore nearby venues

<b> Function to get nearby venues matching the "categories" given place </b>

In [16]:
def getNearbyVenues(names, latitudes, longitudes,url_link,categories, radius):
    ''' Create the venue search url and lookup nearby venues and return as dataframe'''
    venues_list=[]
    for i in range(len(names)):
        name=names[i]
        lat=latitudes[i]
        lng = longitudes[i]
        for category in categories:    
            # create the API request URL
            url = url_link.format(
                category,
                CLIENT_ID, 
                CLIENT_SECRET, 
                lat, 
                lng, 
                VERSION,
                radius,
                LIMIT)
            # make the GET request
            results = requests.get(url).json()    
            print(results)
            try:
                # return only relevant information for each nearby venue
                for j in range(len(results['response']['venues'])):
                        venues_list.append([
                        name, 
                        results['response']['venues'][j]['name'],
                        results['response']['venues'][j]['id'],
                        results['response']['venues'][j]['location']['lat'], 
                        results['response']['venues'][j]['location']['lng'],  
                        results['response']['venues'][j]['categories'][0]['name'],
                        category, 
                        results['response']['venues'][j]['location']['distance'],
                        ])
            except Exception as E:
                    print(E)
        
    nearby_venues = pd.DataFrame(venues_list)
    nearby_venues.columns = ['Address',
              'Venue', 
              'Venue_id',
              'Venue Latitude', 
              'Venue Longitude', 
              'Venue Category',
              'Category ID',
              'Distance [m]']
        
    return(nearby_venues)

def getVenueLikes(venue_ids):
    ''' Obtain the list of number of likes for the venues in "venue_ids"'''
    likes_list=[]
    for i in range(len(venue_ids)): 
        # create the API request URL
        url_link='https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'
        url = url_link.format(venue_ids[i],
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION)


        # make the GET request
        results = requests.get(url).json()
        
        likes_list.append(results['response']['venue']['likes']['count'])

       
    return(likes_list)

Dataframe of Venues for each place, matching the "categories"

In [17]:
# Implement 'getNearbyVenues'
loc_venues=getNearbyVenues(df_loc['Name'], df_loc['Latitude'],df_loc['Longitude'],url_str,categories,radius)
loc_venues.head()

{'meta': {'code': 200, 'requestId': '66a79e24c3d9b85b375e0c65'}, 'response': {'venues': []}}
{'meta': {'code': 200, 'requestId': '66a79e2426d4203fd8d64b6c'}, 'response': {'venues': []}}
{'meta': {'code': 200, 'requestId': '66a79e24d0e24146dad94922'}, 'response': {'venues': []}}
{'meta': {'code': 200, 'requestId': '66a79e257156cb0115c9691b'}, 'response': {'venues': []}}
{'meta': {'code': 200, 'requestId': '66a79e251884e101a6ec59fe'}, 'response': {'venues': []}}


ValueError: Length mismatch: Expected axis has 0 elements, new values have 8 elements

## Pre-Processing

Drop NaN values, set 'Venue' as index column since we are dealing with venues.

In [ ]:
loc_venues.set_index('Venue',inplace=True)
loc_venues.dropna(inplace=True)
loc_venues.shape

Print number of categories for each place

In [ ]:
for i in range(len(places)):
    print('There are {} uniques categories (including subcategories) for '
          .format(len(loc_venues.loc[loc_venues['Address']==places[i],'Venue Category'].unique()))+places[i])

## Exploratory Data Analysis

In [ ]:
# Calculate centroid of the venues for each place in loc_venues
loc_venues_centroids=centroids(loc_venues)
print(loc_venues_centroids)

### Make Folium plot to show venues

In [ ]:
maps={}
loc_results_lat=[]
loc_results_long=[]
zip_results={}

i=0
for place in places:
    # create map
    clustered=loc_venues[loc_venues['Address']==place]
    lat_array=clustered['Venue Latitude']
    long_array=clustered['Venue Longitude']
    venue_name=clustered.index
    
    # Calculate mean latitude and longitude
    latitude=lat_array.mean()                            
    longitude=long_array.mean()
    
    # Update results latitude and longitude arrays
    loc_results_lat.append(latitude)
    loc_results_long.append(longitude)
    
    # Obtain Zipcode
    url_zip=url_str_zip.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION,500,1)
    zip_result=requests.get(url_zip).json()
    try:
        zip_results[place]=zip_result['response']['venues'][0]['location']['formattedAddress']
    except:
        zip_results[place]='0'
    
    print('Centroid for '+str(place)+' at: '+str(round(latitude,5))+', '+str(round(longitude,5))
          +', Address:',zip_results[place][0])

    map_clusters = folium.Map(location=[latitude, longitude],zoom_start=10)

    # add markers to the map
    markers_colors = []
    for lat, lon, name in zip(lat_array, long_array, 
                                 venue_name     ):
        label = folium.Popup(name, parse_html=True)
        folium.Marker(
            [lat, lon],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.7,
                            ).add_to(map_clusters)
        
    folium.RegularPolygonMarker(location=[latitude, longitude], 
                                number_of_sides=5,
                                popup='Centroid', 
                                color='red', 
                                fill=True,
                                fill_color='yellow',
                                fill_opacity=0.8,
                                radius=20).add_to(map_clusters)
    
    maps[place]=map_clusters
    i=i+1

lat1=latitude
long1=longitude
maps[places[0]]


### Make Venue Longitude and Latitude box plots

In [ ]:
fnum=1

unique_cat=len(loc_venues['Category ID'].unique())+1
bp={}                                                   # Box plot object dict.

for i in range(0,unique_cat):
    plt.figure()
    plt.subplot(1,5,1)
    Y=loc_venues.loc[loc_venues['Category ID']==categories[i],'Venue Latitude']
    bp[categories[i]+'.Latitude']=(plt.boxplot(Y))
    plt.xlabel('Latitude')
    plt.xticks([])
    plt.title(str(cat_dict[categories[i]]))

    plt.subplot(1,5,5)
    Y=loc_venues.loc[loc_venues['Category ID']==categories[i],'Venue Longitude']
    bp[categories[i]+'.Longitude']=(plt.boxplot(Y))
    plt.xlabel('Longitude')
    plt.xticks([])
    plt.title(str(cat_dict[categories[i]]))

fnum=fnum+1

### New dataframe for the venues with outliers removed and distance calculated from new centroid

### Geo-distance shift in coordinates

Define function to calculate distance based on geo-coordinates

In [ ]:
def geodistance (coord1,coord2):
    R = 6373.0                          ## Radius of Earth in kms.
    latd1 = math.radians(coord1[0])     # Latitude of coord1 calculated in radians
    lon1 = math.radians(coord1[1])     # Longitude of coord1 calculated in radians
    latd2 = math.radians(coord2[0])     # Latitude of coord2 calculated in radians
    lon2 = math.radians(coord2[1])     # Longitude of coord2 calculated in radians

    dlon = lon2 - lon1
    dlat = latd2 - latd1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c * 1000                    # Geo-distance in kms.
    return distance

## Encoding Venue Categories

In [ ]:
cat_num_dict={cat_id:i for i,cat_id in enumerate(loc_venues['Category ID'].unique())}
loc_venues['Cluster Labels']=loc_venues['Category ID'].apply(lambda x:cat_num_dict[x])
loc_venues.head()

## Display Results

### Generate analyzed map array for each place using folium, and display location for each place

In [ ]:
maps={}
loc_results_lat=[]
loc_results_long=[]
zip_results={}

kclusters=max(loc_venues['Cluster Labels'])+1

for i,place in enumerate(places):
    # create map
    clustered=loc_venues[loc_venues['Address']==place]
    lat_array=clustered['Venue Latitude']
    long_array=clustered['Venue Longitude']
    
    # Calculate mean latitude and longitude
    latitude=loc_venues_new_centroids[place][0]                        
    longitude=loc_venues_new_centroids[place][1] 
    
    # Update results latitude and longitude arrays
    loc_results_lat.append(latitude)
    loc_results_long.append(longitude)
    
    # Obtain Zipcode
    url_zip=url_str_zip.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION,500,1)
    zip_result=requests.get(url_zip).json()
    try:
        zip_results[place]=zip_result['response']['venues'][0]['location']['formattedAddress']
    except:
        zip_results[place]='0'
    
    print('Centroid at: '+str(round(latitude,5))+', '+str(round(longitude,5))
          +', Address:',zip_results[place][0])

    map_clusters = folium.Map(location=[latitude, longitude],zoom_start=10)

    # set color scheme for the clusters
    x = np.arange(kclusters)
    ys = [j + x + (j*x)**2 for j in range(kclusters)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(j) for j in colors_array]

    # add markers to the map
    markers_colors = []
    for lat, lon, poi, cluster, category in zip(lat_array, long_array, 
                                      clustered.index, clustered['Cluster Labels'], clustered['Category ID']):
        label = folium.Popup(str(poi) + ' Cluster: ' + str(cat_dict[category]), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster],
            fill=True,
            fill_color=rainbow[cluster],
            fill_opacity=0.7,
                            ).add_to(map_clusters)
    
    folium.RegularPolygonMarker(location=[latitude, longitude], 
                                number_of_sides=5,
                                popup='Centroid', 
                                color='red', 
                                fill=True,
                                fill_color='yellow',
                                fill_opacity=0.8, 
                                radius=7).add_to(map_clusters)
    
    maps[place]=map_clusters
maps[place]

### Get hotels near venues

In [ ]:
hotel_category=['4bf58dd8d48988d1fa931735']                # Category for Hotels

# Enter Hotel Search URL string
url_str_hotels='https://api.foursquare.com/v2/venues/search?categoryId={}&client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'

df_hotels=df_loc.copy()

df_hotels['Latitude']=[value[0] for value in loc_venues_new_centroids.values()] 
df_hotels['Longitude']=[value[1] for value in loc_venues_new_centroids.values()]
# Get hotels list
hotels=getNearbyVenues(names=df_hotels['Name'], latitudes=df_hotels['Latitude'],longitudes=df_hotels['Longitude'],
                       url_link=url_str_hotels,categories=hotel_category,radius=hotel_radius)

hotel_ids=hotels['Venue_id'].tolist()
# Get Likes list for hotels
likes_list=getVenueLikes(hotel_ids)

hotels['Likes']=likes_list
hotels['Distance [km]']=(hotels['Distance [m]']/1000).astype(int)

hotels.drop(columns=['Category ID','Venue_id','Distance [m]'],inplace=True)
hotels.sort_values(by=['Likes','Distance [km]'],ascending=[False,True],inplace=True)
hotels.set_index('Venue',drop=True,inplace=True)

# Cluster hotels by likes from 1 to 4
likes_clusters=np.arange(1,5)
likes_bracket=np.linspace(0,max(likes_list),5)
hotels['Cluster Labels'] = hotels['Likes'].apply(lambda z: next((
    likes_clusters[i] for i in range(len(likes_clusters)) if likes_bracket[i] <= z < likes_bracket[i + 1]), 
    likes_clusters[-1] if z == likes_bracket[-1] else None))

hotels.drop(columns=['Cluster Labels'])

### Plot map

In [ ]:
hotel_maps={}
zip_results={}
kclusters=max(loc_venues['Cluster Labels'])+1

for i,place in enumerate(places):
    clustered=hotels[hotels.Address==place]

    lat_array=clustered['Venue Latitude']
    long_array=clustered['Venue Longitude']
    
    # Calculate mean latitude and longitude
    latitude=loc_venues_new_centroids[place][0]                           
    longitude=loc_venues_new_centroids[place][1]
        
    # create map
    map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)
    
    # Obtain Zipcode
    url_zip=url_str_zip.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION,500,1)
    zip_result=requests.get(url_zip).json()
    try:
        zip_results[place]=zip_result['response']['venues'][0]['location']['formattedAddress']
    except:
        zip_results[place]='0'
    
    print('Centroid at: ' +str(round(latitude,5))+', '+str(round(longitude,5))
          +',Centroid Address:',zip_results[place][0])

    # set color scheme for the clusters
    x = np.arange(kclusters)
    ys = [j + x + (j*x)**2 for j in range(kclusters)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)+1))
    rainbow = [colors.rgb2hex(j) for j in colors_array]

    # add markers to the map
    markers_colors = []
    for lat, lon, poi, cluster in zip(lat_array, long_array, 
                                    clustered.index, clustered['Cluster Labels']):
        label = folium.Popup(str(poi) + ' Score(' + str(cluster)+'/4): ', parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster],
            fill=True,
            fill_color=rainbow[cluster],
            fill_opacity=0.7,
                            ).add_to(map_clusters)
    folium.RegularPolygonMarker(location=[latitude, longitude], 
                            number_of_sides=5,
                            popup='Centroid', 
                            color='red', 
                            fill=True,
                            fill_color='yellow',
                            fill_opacity=0.8,
                            radius=7).add_to(map_clusters)
    
    hotel_maps[place]=map_clusters
hotel_maps[place]